In [27]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup

r= requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

In [28]:
raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
card_list[0:5]

['Muldrotha, the Gravetide',
 'Abrupt Decay',
 'Acidic Slime',
 "Alchemist's Refuge",
 'Arcane Lighthouse']